<a href="https://colab.research.google.com/github/zebedy-magic/BottleImageRecognition/blob/master/Bottle_recognition_part4_ipynb_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lets try a neural network and 
Lets experiment with our images 
 
 

*   I am going to create a really simple network first to see if a binary classiifcation works
*   Next maybe a CNN convoluted nueral network for more sophisticated recognition



In [96]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import pandas as pd
from os.path import join
from PIL import Image
import numpy as np

#root directory
root='/content/drive/My Drive/'

image_path=join(root,'images','processed')

#test

image_path=join(root,'images','test','processed')

In [66]:
def get_image(filename,image_path):
  #get the image
  img=Image.open(join(image_path,filename))
  return np.array(img)

In [67]:
def get_all_images(labels):
  lst_images=[]
  for image in labels.index:
    img_arr=get_image(image,image_path)
    lst_images.append(img_arr)
    matrix=np.array(lst_images)
  return matrix

In [68]:
def create_matrix(labels):
  #create feature matrix using the creat features 
  lst_images=[]
  #loop through the images .get the features and append to a list
  for image in labels.index:
    img_arr=get_image(image,image_path)
    img_flat=img_arr.flatten()
   
    
    lst_images.append(img_flat)
  #use array to make it into one big matrix
  matrix=np.array(lst_images)
  return matrix

In [69]:
#load the training set 
labels=pd.read_csv(join(root,'labels.csv'),index_col=0)
print(labels.head())
print(labels.info())
target=to_categorical(labels['bottle'])


                                 bottle
image                                  
2020-04-25 12:01:24.739043.jpeg       1
2020-04-25 12:01:26.103276.jpeg       0
2020-04-25 12:01:27.977304.jpeg       0
2020-04-25 12:01:29.199803.jpeg       1
2020-04-25 12:01:30.760437.jpeg       0
<class 'pandas.core.frame.DataFrame'>
Index: 439 entries, 2020-04-25 12:01:24.739043.jpeg to 2020-06-23 08:54:16.864046.jpeg
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   bottle  439 non-null    int64
dtypes: int64(1)
memory usage: 6.9+ KB
None


In [ ]:


matrix=create_matrix(labels)
print(matrix.shape)

ss=StandardScaler()
ss_matrix=ss.fit_transform(matrix)



(439, 40000)


In [ ]:
#lets create a basic dense network with keras and use the processed images 

early_stopping_monitor=EarlyStopping(patience=3)

model= Sequential()

model.add(Dense(200,activation='relu',input_shape=(40000,)))

model.add(Dense(200,activation='relu'))



model.add(Dense(2,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(ss_matrix,target,validation_split=0.2,epochs=30,callbacks=[early_stopping_monitor])

Train on 351 samples, validate on 88 samples
Epoch 1/30
351/351 [==============================] - 1s 4ms/step - loss: 9.4744 - accuracy: 0.7379 - val_loss: 1.0623 - val_accuracy: 0.9432
Epoch 2/30
351/351 [==============================] - 1s 4ms/step - loss: 6.5820 - accuracy: 0.8319 - val_loss: 0.8913 - val_accuracy: 0.9773
Epoch 3/30
351/351 [==============================] - 1s 3ms/step - loss: 1.8686 - accuracy: 0.9231 - val_loss: 0.0408 - val_accuracy: 0.9886
Epoch 4/30
351/351 [==============================] - 1s 3ms/step - loss: 0.8655 - accuracy: 0.9544 - val_loss: 0.3390 - val_accuracy: 0.9773
Epoch 5/30
351/351 [==============================] - 1s 3ms/step - loss: 0.7981 - accuracy: 0.9516 - val_loss: 0.2475 - val_accuracy: 0.9773
Epoch 6/30
351/351 [==============================] - 1s 3ms/step - loss: 0.9784 - accuracy: 0.9288 - val_loss: 0.3527 - val_accuracy: 0.9773


In [94]:
#lets try and evaluate on unknown test set 
#load in test_data
labels_test=pd.read_csv(join(root,'labels_test.csv'),index_col=0)
print(labels_test.head())
print(labels_test.info())
target_test=to_categorical(labels_test['bottle'])



                                 bottle
image                                  
2020-06-23 13:17:46.741908.jpeg       1
2020-06-23 13:17:45.446242.jpeg       1
2020-06-23 13:17:43.266535.jpeg       1
2020-06-23 13:17:44.312072.jpeg       1
2020-06-23 13:17:42.124396.jpeg       1
<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 2020-06-23 13:17:46.741908.jpeg to crap.jpeg
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   bottle  10 non-null     int64
dtypes: int64(1)
memory usage: 160.0+ bytes
None


In [97]:


matrix_test=create_matrix(labels_test)
print(matrix_test.shape)


ss_matrix_test=ss.fit_transform(matrix_test)



(10, 40000)


In [ ]:
model.evaluate(ss_matrix_test,target_test)

10/10 [==============================] - 0s 1ms/step


[2.1252803802490234, 0.8999999761581421]

lets use a CNN and see if this is better 


In [57]:
from keras.layers import Conv2D,Flatten


In [91]:
#get images not flattened 
def get_matrix(labels):
  matrix_3d=get_all_images(labels)
  matrix=np.reshape(matrix_3d,(matrix_3d.shape[0],200,200,1))
  
  return matrix

In [82]:
matrix=get_matrix(labels_test)
print(matrix.shape)

model=Sequential()
model.add(Conv2D(10,kernel_size=3,activation='relu',input_shape=(200,200,1)))
model.add(Flatten())
model.add(Dense(2,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(matrix,target,validation_split=0.3,epochs=30,callbacks=[early_stopping_monitor])

(439, 200, 200, 1)
Train on 307 samples, validate on 132 samples
Epoch 1/30
307/307 [==============================] - 3s 11ms/step - loss: 1913.6351 - accuracy: 0.5342 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/30
307/307 [==============================] - 3s 11ms/step - loss: 1722.3876 - accuracy: 0.5114 - val_loss: 2316.9906 - val_accuracy: 0.0000e+00
Epoch 3/30
307/307 [==============================] - 3s 11ms/step - loss: 561.3417 - accuracy: 0.5863 - val_loss: 56.3857 - val_accuracy: 0.8182
Epoch 4/30
307/307 [==============================] - 3s 11ms/step - loss: 255.8408 - accuracy: 0.7296 - val_loss: 9.2462 - val_accuracy: 0.9773


In [98]:
matrix_test=get_matrix(labels_test)
model.evaluate(matrix_test,target_test)

10/10 [==============================] - 0s 6ms/step


[79.42049407958984, 0.8999999761581421]